In [10]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
import pandas as pd
import os
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import LogisticRegression
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, OneHotEncoder, OrdinalEncoder, MinMaxScaler
from sklearn.model_selection import ParameterGrid
from sklearn.metrics import make_scorer
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.metrics import r2_score
from sklearn.model_selection import StratifiedGroupKFold
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import make_pipeline

warnings.simplefilter(action='ignore', category=FutureWarning)
import xgboost
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
from sklearn.utils.class_weight import compute_sample_weight
from sklearn.metrics import confusion_matrix

from sklearn import metrics
from sklearn.model_selection import GroupKFold
from sklearn.model_selection import GroupShuffleSplit

In [3]:
os.chdir('../data')


X = pd.read_csv('X_3specialties_equalWeight_subsample.zip',compression='zip', index_col=False)
y = pd.read_csv('y_3specialties_equalWeight_subsample.zip',compression='zip')
groups = pd.read_csv('groups_3specialties_equalWeight_subsample.zip',compression='zip')

X = X.iloc[:,1:]
y = y.iloc[:,1:]
groups = groups.iloc[:,1:]

y_columns = y.columns

le = LabelEncoder()
y = y.values.ravel()
y = le.fit_transform(y)
y = pd.DataFrame(y)
y.columns = y_columns

In [16]:
len(y)*0.8*0.2

672.6400000000001

In [4]:
categorical_ftrs = ['Prscrbr_City',
                    'Prscrbr_State_Abrvtn',
                    'Brnd_Name',
                    'Gnrc_Name']

std_ftrs = ['Tot_Clms', 
            'Tot_30day_Fills', 
            'Tot_Day_Suply', 
            'Tot_Drug_Cst', 
            'Tot_Benes', 
            'GE65_Tot_Clms',
            'GE65_Tot_30day_Fills',
            'GE65_Tot_Drug_Cst',
            'GE65_Tot_Day_Suply',
            'GE65_Tot_Benes']

preprocessor = ColumnTransformer(
    transformers=[
        ('onehot', OneHotEncoder(sparse=False,handle_unknown='ignore'), categorical_ftrs),
        ('std', StandardScaler(), std_ftrs)])


In [5]:
X.isnull()

,Prscrbr_City,Prscrbr_State_Abrvtn,Brnd_Name,Gnrc_Name,Tot_Clms,Tot_30day_Fills,Tot_Day_Suply,Tot_Drug_Cst,Tot_Benes,GE65_Tot_Clms,GE65_Tot_30day_Fills,GE65_Tot_Drug_Cst,GE65_Tot_Day_Suply,GE65_Tot_Benes
0,False,False,False,False,False,False,False,False,True,True,True,True,True,True
1,False,False,False,False,False,False,False,False,True,False,False,False,False,True
2,False,False,False,False,False,False,False,False,True,False,False,False,False,True
3,False,False,False,False,False,False,False,False,False,False,False,False,False,False
4,False,False,False,False,False,False,False,False,True,False,False,False,False,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4199,False,False,False,False,False,False,False,False,True,False,False,False,False,True
4200,False,False,False,False,False,False,False,False,False,True,True,True,True,True
4201,False,False,False,False,False,False,False,False,True,False,False,False,False,True
4202,False,False,False,False,False,False,False,False,False,True,True,True,True,True


In [6]:
mask = X.isnull()

unique_rows, counts = np.unique(mask, axis=0, return_counts=True)
print(unique_rows.shape) # 5 patterns, we will train 6 models

for i in range(len(counts)):
    print(unique_rows[i],counts[i])

(5, 14)
[False False False False False False False False False False False False
 False False] 441
[False False False False False False False False False False False False
 False  True] 587
[False False False False False False False False False  True  True  True
  True  True] 811
[False False False False False False False False  True False False False
 False  True] 1303
[False False False False False False False False  True  True  True  True
  True  True] 1062


In [90]:
def xgb_model(X_train, Y_train, X_CV, y_CV, X_test, y_test, i, verbose=4):

    # make into row vectors to avoid an obnoxious sklearn/xgb warning
    Y_train = np.reshape(np.array(Y_train), (1, -1)).ravel()
    y_CV = np.reshape(np.array(y_CV), (1, -1)).ravel()
    y_test = np.reshape(np.array(y_test), (1, -1)).ravel()

    XGB = xgboost.XGBClassifier(num_class=3,
                                objective = "multi:softprob",
                                eval_metric = 'mlogloss', 
                                random_state = i, 
                                use_label_encoder = False)
    
    # find the best parameter set
    param_grid = {"learning_rate": [0.03],#0.01,
                  "n_estimators": [10000],
                  "seed": [i],
                  "reg_alpha": [0e0, 1e-2, 1e-1, 1e0, 1e1, 1e2],
                  #"reg_lambda": [0e0, 1e-2, 1e-1, 1e0, 1e1, 1e2],
                  "missing": [np.nan], 
                  "max_depth": [3], #1,3,
                  "colsample_bytree": [0.7],              
                  "subsample": [0.7]} #, 0.9

    pg = ParameterGrid(param_grid)

    scores_f1 = np.zeros(len(pg))

    #weights = compute_sample_weight(class_weight='balanced', y = Y_train)
    
    for i in range(len(pg)):
        if verbose >= 5:
            print("Param set " + str(i + 1) + " / " + str(len(pg)))
        params = pg[i]
        XGB.set_params(**params)
        eval_set = [(X_CV, y_CV)]
        XGB.fit(X_train, Y_train,
                early_stopping_rounds=50, eval_set=eval_set, verbose=False)
                #sample_weight = weights)# with early stopping
        y_CV_pred = XGB.predict(X_CV, iteration_range=(0, XGB.best_ntree_limit))
        #scores[i] = accuracy_score(y_CV,y_CV_pred)
        scores_f1[i] = f1_score(y_CV,y_CV_pred, average = 'macro')

    best_params = np.array(pg)[scores_f1 == np.max(scores_f1)]
    if verbose >= 4:
        print('Test set max score and best parameters are:')
        print(np.max(scores_f1))
        print(best_params)
        print()
    # test the model on the test set with best parameter set
    XGB.set_params(**best_params[0])
    XGB.fit(X_train,Y_train,
            early_stopping_rounds=50,eval_set=eval_set, verbose=False)
    y_test_pred = XGB.predict(X_test, iteration_range=(0, XGB.best_ntree_limit))

    if verbose >= 1:
        print ('The CV F1 is:',f1_score(y_CV,y_CV_pred, average = 'macro'))
        print()
    if verbose >= 2:
        print ('The predictions are:')
        print (y_test_pred)
        print()
    #if verbose >= 3:
        #print("Feature importances:")
        #print(XGB.feature_importances_)
    
    return (f1_score(y_test,y_test_pred, average='macro'), y_test_pred, XGB.feature_importances_)
    #return (y_test_pred, best_params, XGB.feature_importances_ )


In [91]:
# Function: Reduced-feature XGB model
# all the inputs need to be pandas DataFrame
def reduced_feature_xgb(X_train, Y_train, X_CV, y_CV, X_test, y_test, i):
    
    # find all unique patterns of missing value in test set
    mask = X_test.isnull()
    unique_rows = np.array(np.unique(mask, axis=0))
    all_y_test_pred = pd.DataFrame()
    
    print('    There are', len(unique_rows), 'unique missing value patterns.')
    print()
    # divide test sets into subgroups according to the unique patterns
    for i in range(len(unique_rows)):
        print()
        print ('    *** Working on unique pattern', i, ' ***')
        print()
        ## generate X_test subset that matches the unique pattern i
        sub_X_test = pd.DataFrame()
        sub_y_test = pd.Series(dtype=float)
        for j in range(len(mask)): # check each row in mask
            row_mask = np.array(mask.iloc[j])
            if np.array_equal(row_mask, unique_rows[i]): # if the pattern matches the ith unique pattern
                sub_X_test = sub_X_test.append(X_test.iloc[j])# append the according X_test row j to the subset
                sub_y_test = sub_y_test.append(y_test.iloc[j])# append the according y_test row j
                                                
        sub_X_test = sub_X_test[X_test.columns[~unique_rows[i]]]
        
        ## choose the according reduced features for subgroups
        sub_X_train = pd.DataFrame()
        sub_Y_train = pd.DataFrame()
        sub_X_CV = pd.DataFrame()
        sub_y_CV = pd.DataFrame()
        # 1.cut the feature columns that have nans in the according sub_X_test
        sub_X_train = X_train[X_train.columns[~unique_rows[i]]]
        sub_X_CV = X_CV[X_CV.columns[~unique_rows[i]]]
        # 2.cut the rows in the sub_X_train and sub_X_CV that have any nans
        sub_X_train = sub_X_train.dropna()
        sub_X_CV = sub_X_CV.dropna()   
        # 3.cut the sub_Y_train and sub_y_CV accordingly
        sub_Y_train = Y_train.iloc[sub_X_train.index]
        sub_y_CV = y_CV.iloc[sub_X_CV.index]
        
        # run XGB
        sub_y_test_pred = xgb_model(sub_X_train, sub_Y_train, sub_X_CV, 
                                       sub_y_CV, sub_X_test, sub_y_test, i, verbose=4)
        sub_y_test_pred = pd.DataFrame(sub_y_test_pred[1],columns=['sub_y_test_pred'],
                                          index=sub_y_test.index)
        print()
        print('   Accuracy:', accuracy_score(sub_y_test,sub_y_test_pred))
        print('   F1 Score:', f1_score(sub_y_test, sub_y_test_pred, average = 'macro' ))
        
        # collect the test predictions
        all_y_test_pred = all_y_test_pred.append(sub_y_test_pred)
        
    # rank the final y_test_pred according to original y_test index
    all_y_test_pred = all_y_test_pred.sort_index()
    y_test = y_test.sort_index()
               
    # get global scores
    total_accuracy = (accuracy_score(y_test,all_y_test_pred))
    f1 = f1_score(y_test,all_y_test_pred, average = 'macro')
    
    cm = confusion_matrix
    
    return total_accuracy, f1, cm

In [92]:

def ML_pipeline_ReducedFeatXGBoost_GridSearchCV(X, y, groups, i):
    # create a test set based on groups
    splitter = GroupShuffleSplit(n_splits=1,test_size=0.2,random_state=i)
    
    # Get Test Set
    for i_other,i_test in splitter.split(X, y, groups):
        X_other, y_other, groups_other = X.iloc[i_other], y.iloc[i_other], groups.iloc[i_other]
        X_test, y_test, groups_test = X.iloc[i_test], y.iloc[i_test], groups.iloc[i_test]
    
    # Get Validation Set
    
    kf = GroupKFold(n_splits=5)
    counter = 0
    for i_train, i_test in kf.split(X_other, y_other, groups_other):
        X_train, y_train, groups_train = X_other.iloc[i_train], y_other.iloc[i_train], groups_other.iloc[i_train]
        X_val, y_val, groups_val = X_other.iloc[i_test], y_other.iloc[i_test], groups_other.iloc[i_test]
        
        #print(len(y_val))
        #print(len(y_train))
        counter = counter + 1
        
        print(f"CV # {counter}")
        X_prep = preprocessor.fit_transform(X_train)
        # collect feature names
        
        feature_names = preprocessor.get_feature_names_out()
        
        df_train = pd.DataFrame(data=X_prep,columns=feature_names)
        print(f"Train Set Shape (after preprocessing): {df_train.shape}")
        print()
        
        # transform the CV
        df_CV = preprocessor.transform(X_val)
        df_CV = pd.DataFrame(data=df_CV,columns = feature_names)
        print(f"CV Set Shape (after preprocessing): {df_CV.shape}")
        print()
        # transform the test
        df_test = preprocessor.transform(X_test)
        df_test = pd.DataFrame(data=df_test,columns = feature_names)
        print(f"Test Set Shape (after preprocessing): {df_test.shape}")
        print()
        
        y_CV = y_val
        
        total_accuracy, f1, cm = reduced_feature_xgb(df_train, y_train, df_CV, y_CV, df_test, y_test, i)
        
        return total_accuracy, f1, cm

In [93]:

%%time
acc_scores = []
f1_scores = []
best_grid_params = []
confusion_mat = []
#class_met = []

for i in range(1):
    print(f'Random State # {i}')
    print()
    
    total_accuracy, f1, cm = ML_pipeline_ReducedFeatXGBoost_GridSearchCV(X, y, groups, i)
    
    confusion_mat.append(cm)
    acc_scores.append(total_accuracy)
    #class_met.append(class_metrics)
    
    #print(grid.best_params_)
    
    #best_grid_params.append(best_params)
    #print()
    #print('best CV score:',grid.best_score_)
    #print()
    print('f1 score:', f1)
    f1_scores.append(f1)
    print()
    
print('test score:',np.around(np.mean(f1_scores),2),'+/-',np.around(np.std(f1_scores),2))


print('test score:',np.around(np.mean(acc_scores),2),'+/-',np.around(np.std(acc_scores),2))



Random State # 0

CV # 1
Train Set Shape (after preprocessing): (2808, 856)

CV Set Shape (after preprocessing): (702, 856)

Test Set Shape (after preprocessing): (694, 856)

    There are 5 unique missing value patterns.


    *** Working on unique pattern 0  ***

Test set max score and best parameters are:
0.8346894415911671
[{'colsample_bytree': 0.7, 'learning_rate': 0.03, 'max_depth': 3, 'missing': nan, 'n_estimators': 10000, 'reg_alpha': 1.0, 'seed': 0, 'subsample': 0.7}]

The CV F1 is: 0.18939393939393942

The predictions are:
[0 0 0 2 0 2 0 0 2 1 0 2 0 0 1 1 1 1 1 1 2 2 2 0 0 0 0 0 0 0 2 2 2 2 2 0 0
 2 2 2 0 2 2 2 2 2 2 2 2 2 2 0 0 0 0 2 2 2 2 2 2 2 2 0 0 0 2 2 2 2 2 2 2 2
 2 2 2 2 2 0 0 2 0 2 2 2 2 2 2 2 2 2 2 2 2]


   Accuracy: 0.8210526315789474
   F1 Score: 0.74022281590323

    *** Working on unique pattern 1  ***

Test set max score and best parameters are:
0.7888722998409466
[{'colsample_bytree': 0.7, 'learning_rate': 0.03, 'max_depth': 3, 'missing': nan, 'n_estimators':

In [89]:
acc_scores

[0.3746397694524496]